In [1]:
import numpy as np
from matplotlib import pyplot as plt
import json
from pymongo import MongoClient
client = MongoClient()
db = client['gtr']

In [2]:
repo_targets = {}
for doc in db['repo_core_targets'].find():
    repo_targets[doc['repo']] = doc['target_teams']

In [3]:
def NDCG(repo,rec,repo_targets):
    dcg = 0
    for i,tm in enumerate(rec):
        if tm in repo_targets[repo]:
            dcg += 1/np.log2(i+2)
    target_cnt = 0
    idcg =  0
    for tm in repo_targets[repo]:
        target_cnt += 1
        idcg += 1/np.log2(target_cnt+1)
    dcg /= idcg
    return dcg

In [4]:
recommendations = {}
with open('sae_team_score_contri.json') as rj:
    for l in rj.readlines():
        line = l.strip().split('\t')
        repo = line[0]
        recs = [json.loads(r)[0] for r in line[1:]]
        recommendations[repo] = recs

In [6]:
NDCGs = []
for repo in recommendations:
    if not repo in repo_targets:
        continue
    NDCGs.append(NDCG(repo,recommendations[repo],repo_targets))
plt.hist(NDCGs)
plt.xlabel("nDCG")
plt.title("Evaluation of SAE-Based FunkSVD (Weight by Contribution)")
plt.savefig('evaluate_interest_mean.jpg',bbox='tight')
plt.show()
print(min(NDCGs),max(NDCGs))

ZeroDivisionError: division by zero

In [ ]:
recommendations = {}
with open('sae_team_score_degree.json') as rj:
    for l in rj.readlines():
        line = l.strip().split('\t')
        repo = line[0]
        recs = [json.loads(r)[0] for r in line[1:]]
        recommendations[repo] = recs

In [ ]:
NDCGs = []
for repo in recommendations:
    NDCGs.append(NDCG(repo,recommendations[repo],repo_targets))
plt.hist(NDCGs)
plt.xlabel("nDCG")
plt.title("Evaluation of SAE-Based FunkSVD (Weighted by Degree)")
plt.savefig('evaluate_interest_contri.jpg',bbox='tight')
plt.show()
print(min(NDCGs),max(NDCGs))

In [ ]:
recommendations = {}
with open('sae_team_score_min.json') as rj:
    for l in rj.readlines():
        line = l.strip().split('\t')
        repo = line[0]
        recs = [json.loads(r)[0] for r in line[1:]]
        recommendations[repo] = recs

In [ ]:
NDCGs = []
for repo in recommendations:
    NDCGs.append(NDCG(repo,recommendations[repo],repo_targets))
plt.hist(NDCGs)
plt.xlabel("nDCG")
plt.title("Evaluation of Interest-Based Recommendation (Min Interest)")
plt.savefig('evaluate_interest_degree.jpg',bbox='tight')
plt.show()
print(min(NDCGs),max(NDCGs))

In [ ]:
recommendations = {}
with open('sae_team_score_max.json') as rj:
    for l in rj.readlines():
        line = l.strip().split('\t')
        repo = line[0]
        recs = [json.loads(r)[0] for r in line[1:]]
        recommendations[repo] = recs

In [ ]:
NDCGs = []
for repo in recommendations:
    NDCGs.append(NDCG(repo,recommendations[repo],repo_targets))
plt.hist(NDCGs)
plt.xlabel("nDCG")
plt.title("Evaluation of SAE-Based FunkSVD (Max Interest)")
plt.savefig('evaluate_interest_degree.jpg',bbox='tight')
plt.show()
print(min(NDCGs),max(NDCGs))

In [ ]:
recommendations = {}
with open('sae_team_score_mean.json') as rj:
    for l in rj.readlines():
        line = l.strip().split('\t')
        repo = line[0]
        recs = [json.loads(r)[0] for r in line[1:]]
        recommendations[repo] = recs

In [ ]:
NDCGs = []
for repo in recommendations:
    NDCGs.append(NDCG(repo,recommendations[repo],repo_targets))
plt.hist(NDCGs)
plt.xlabel("nDCG")
plt.title("Evaluation of SAE-Based FunkSVD (Mean Interest)")
plt.savefig('evaluate_interest_degree.jpg',bbox='tight')
plt.show()
print(min(NDCGs),max(NDCGs))